In [ ]:
#Load and Explore data
import pandas as pd # type: ignore

df= pd.read_csv(r"C:\Users\sarvesh\OneDrive\文档\Desktop\Telo_customer_churn.csv")
df.head()
df.isnull().sum()

#data preprocessing
df.drop('customerID',axis=1, inplace=True)
df['TotalCharges']= pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(inplace=True)
from sklearn.preprocessing import LabelEncoder # type: ignore
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

#feature scaling
from sklearn.preprocessing import StandardScaler # type: ignore
x= df.drop('Churn', axis=1)
y= df['Churn']
scaler= StandardScaler()
x_scaled= scaler.fit_transform(x)

#Model Training
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.ensemble import RandomForestClassifier # type: ignore
from sklearn.metrics import classification_report, confusion_matrix # type: ignore
x_train, x_test, y_train, y_test= train_test_split(x_scaled, y, test_size=0.2, random_state=42)
model= RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)
y_pred= model.predict(x_test)
print("classification Report")
print(classification_report(y_test, y_pred))
print("confusion Matrix")
print(confusion_matrix(y_test, y_pred))

#customer segmentation
df['Prediction'] = model.predict(x_scaled)
def segment_customer(row):
    if row['Prediction'] == 1 and row['MonthlyCharges'] > df['MonthlyCharges'].quantile(0.75):
        return 'At Risk'
    elif row['Prediction'] == 0 and row['MonthlyCharges'] < df['MonthlyCharges'].quantile(0.25):
        return 'Loyal'
    else:
        return 'Dormant'
df['Segment'] = df.apply(segment_customer, axis=1)
print(df['Segment'].value_counts())

# Save the final dataframe to a CSV file
df.to_csv('Telco_Customer_Churn_Segmented.csv', index=False)
print("Segmented data saved to 'Telco_Customer_Churn_Segmented.csv'")






classification Report
              precision    recall  f1-score   support

           0       0.82      0.91      0.86      1033
           1       0.64      0.46      0.53       374

    accuracy                           0.79      1407
   macro avg       0.73      0.68      0.70      1407
weighted avg       0.77      0.79      0.77      1407

confusion Matrix
[[935  98]
 [203 171]]
Segment
Dormant    4905
Loyal      1576
At Risk     551
Name: count, dtype: int64
Segmented data saved to 'Telco_Customer_Churn_Segmented.csv'
